In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1.post2
Torchvision Version:  0.2.1


In [2]:
from PIL import Image 
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
trainLabels = pd.read_csv("./trainLabels.csv")

In [3]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "./train/train_224"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 5

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 10

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

In [4]:
train_path = './train/train_224/*'
images_list = glob.glob(train_path)
images_list = sorted(images_list)

In [5]:
# number_of_images = 10
# def createDataset(images_list,trainLabels):
#     images = []
#     imlabel = []
#     for img in images_list[:number_of_images]:
#         images.append(np.array(Image.open(img)))
#         fileName = img.split('/')[-1].split('.')[0]
#         imlabel_category = np.zeros(5)
#         imlabel_category[trainLabels.loc[trainLabels.image==fileName, 'level'].values[0]] = 1
#         imlabel.append(imlabel_category)
#     images = np.array(images)
#     imlabel = np.array(imlabel)
#     return images , imlabel

In [6]:
# X , y = createDataset(images_list,trainLabels)

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [8]:
# X_train = torch.from_numpy(X_train)
# y_train = torch.from_numpy(y_train)
# X_test = torch.from_numpy(X_test)
# y_test = torch.from_numpy(y_test)

In [9]:
class EyeDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, images_list,trainLables,transform=None):
#         self.X = X
#         self.y = y
#         self.transform = transforms.Resize(224)
        self.images_list = images_list
        self.trainLables = trainLables
        self.transform = transform

    def __len__(self):
        print(len(self.images_list))
        return len(self.images_list)

    def __getitem__(self, idx):
#         print("index ",idx)
        imgtemp = Image.open(self.images_list[idx])
        if self.transform!=None :
            imgtemp = self.transform(imgtemp)
        img = transforms.ToTensor()(imgtemp)
        fileName = self.images_list[idx].split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[self.trainLables.loc[self.trainLables.image==fileName ,'level'].values[0]] = 1
        
#         xtemp = self.transform(self.X[idx])
#         ytemp = self.transform(self.y[idx])
        return img ,imglabel

class EyeDatasetFromImages(torch.utils.data.Dataset):
    
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]

In [7]:
images_train_list ,images_test_list = train_test_split(images_list,test_size=0.2,random_state = 4)

In [11]:
# dataloaders_dict = { 'train' : torch.utils.data.DataLoader(EyeDatasetFromImages(X_train,y_train),batch_size=batch_size) }
# dataloaders_dict['val'] = torch.utils.data.DataLoader(EyeDatasetFromImages(X_test,y_test),batch_size=batch_size)

In [12]:
dataloaders_dict = { 'train' : torch.utils.data.DataLoader(EyeDataset(images_train_list,trainLabels),batch_size=batch_size) }
dataloaders_dict['val'] = torch.utils.data.DataLoader(EyeDataset(images_test_list,trainLabels),batch_size=batch_size)

In [13]:
def train_model(model, dataloaders, criterion, optimizer, checkpoint,num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            num_b = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                num_b += 1
                inputs = inputs.to(device)
                labels = labels.to(device)
#                 print("input size ",inputs.size())
#                 print("label size",labels.size())
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, torch.max(labels,1)[1])

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == torch.max(labels,1)[1])
                
                if num_b%35==0 :
                    print('loss in {} batch is {:.4f}'.format(num_b,loss))
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, checkpoint)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [14]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [15]:
def toy_model():
    return models.vgg19_bn(pretrained=True)

In [16]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    return model_ft , input_size

In [17]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# model_toy = toy_model()
# model_toy.classifier[6] = nn.Linear(4096,num_classes)
checkpoint = "resnet50.checkpoint"

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
model_ft = model_ft.to(device)
# model_toy = model_toy.to(device)

In [20]:
params_to_update = model_ft.parameters()
# params_to_update = model_toy.parameters()


In [21]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [26]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,
                             "resnet50_12epoch.checkpoint", num_epochs=num_epochs, is_inception=(model_name=="inception"))
# model_toy, hist = train_model(model_toy, dataloaders_dict, criterion, optimizer_ft, 
#                               checkpoint,num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/9
----------
28100
loss in 35 batch is 0.1789
loss in 70 batch is 0.3338
loss in 105 batch is 0.3331
loss in 140 batch is 0.0903
loss in 175 batch is 1.4743
loss in 210 batch is 0.3177
loss in 245 batch is 1.3052
loss in 280 batch is 0.6386
loss in 315 batch is 0.3779
loss in 350 batch is 0.2563
loss in 385 batch is 0.1926
loss in 420 batch is 0.2652
loss in 455 batch is 2.1361
loss in 490 batch is 0.4513
loss in 525 batch is 0.3772
loss in 560 batch is 0.7516
loss in 595 batch is 0.5935
loss in 630 batch is 0.3800
loss in 665 batch is 0.5735
loss in 700 batch is 0.4593
loss in 735 batch is 0.7562
loss in 770 batch is 0.1595
loss in 805 batch is 0.6343
loss in 840 batch is 0.4571
loss in 875 batch is 0.3624
loss in 910 batch is 1.1375
loss in 945 batch is 0.6143
loss in 980 batch is 0.8383
loss in 1015 batch is 0.5657
loss in 1050 batch is 0.5734
loss in 1085 batch is 0.9962
loss in 1120 batch is 0.8102
loss in 1155 batch is 0.7825
loss in 1190 batch is 0.3564
loss in 1225 batc

loss in 1050 batch is 0.2884
loss in 1085 batch is 0.4522
loss in 1120 batch is 0.3813
loss in 1155 batch is 0.3633
loss in 1190 batch is 0.7089
loss in 1225 batch is 0.6529
loss in 1260 batch is 0.0824
loss in 1295 batch is 0.6711
loss in 1330 batch is 0.2329
loss in 1365 batch is 1.1268
loss in 1400 batch is 0.3369
loss in 1435 batch is 0.7596
loss in 1470 batch is 0.2798
loss in 1505 batch is 0.4747
loss in 1540 batch is 0.1766
loss in 1575 batch is 0.4280
loss in 1610 batch is 0.8317
loss in 1645 batch is 0.2391
loss in 1680 batch is 0.5607
loss in 1715 batch is 0.4083
loss in 1750 batch is 0.1516
loss in 1785 batch is 0.3488
loss in 1820 batch is 0.1136
loss in 1855 batch is 0.3979
loss in 1890 batch is 0.5445
loss in 1925 batch is 0.3802
loss in 1960 batch is 1.3015
loss in 1995 batch is 0.5171
loss in 2030 batch is 0.8531
loss in 2065 batch is 0.4650
loss in 2100 batch is 0.5465
loss in 2135 batch is 0.1783
loss in 2170 batch is 0.4274
loss in 2205 batch is 0.6838
loss in 2240 b

loss in 2065 batch is 0.6153
loss in 2100 batch is 0.2590
loss in 2135 batch is 0.1000
loss in 2170 batch is 0.1725
loss in 2205 batch is 0.3837
loss in 2240 batch is 0.0793
loss in 2275 batch is 0.2871
loss in 2310 batch is 0.3504
loss in 2345 batch is 0.1811
loss in 2380 batch is 0.1371
loss in 2415 batch is 0.1048
loss in 2450 batch is 0.1579
loss in 2485 batch is 0.1213
loss in 2520 batch is 0.2082
loss in 2555 batch is 0.2396
loss in 2590 batch is 0.5573
loss in 2625 batch is 0.3212
loss in 2660 batch is 0.7371
loss in 2695 batch is 0.1142
loss in 2730 batch is 0.5792
loss in 2765 batch is 0.6485
loss in 2800 batch is 0.6914
loss in 2835 batch is 0.3254
loss in 2870 batch is 0.1818
loss in 2905 batch is 0.1503
loss in 2940 batch is 0.3467
loss in 2975 batch is 0.3286
loss in 3010 batch is 0.1244
loss in 3045 batch is 0.0516
loss in 3080 batch is 0.2303
loss in 3115 batch is 0.1224
loss in 3150 batch is 0.2072
loss in 3185 batch is 0.6988
loss in 3220 batch is 0.7992
loss in 3255 b

loss in 3080 batch is 0.3316
loss in 3115 batch is 0.0801
loss in 3150 batch is 0.0837
loss in 3185 batch is 0.2358
loss in 3220 batch is 0.1733
loss in 3255 batch is 0.3165
loss in 3290 batch is 0.2255
loss in 3325 batch is 0.0970
loss in 3360 batch is 0.4588
loss in 3395 batch is 0.0620
loss in 3430 batch is 0.7692
loss in 3465 batch is 0.0789
loss in 3500 batch is 0.0295
28100
28100
train Loss: 0.2686 Acc: 0.9010
7026
loss in 35 batch is 1.1376
loss in 70 batch is 0.4889
loss in 105 batch is 1.4517
loss in 140 batch is 0.1982
loss in 175 batch is 2.7226
loss in 210 batch is 0.9896
loss in 245 batch is 0.2470
loss in 280 batch is 1.6799
loss in 315 batch is 0.7143
loss in 350 batch is 0.8083
loss in 385 batch is 0.5226
loss in 420 batch is 0.5047
loss in 455 batch is 2.8197
loss in 490 batch is 0.3080
loss in 525 batch is 0.4440
loss in 560 batch is 0.8782
loss in 595 batch is 1.0066
loss in 630 batch is 1.3565
loss in 665 batch is 1.9682
loss in 700 batch is 1.1440
loss in 735 batch

loss in 560 batch is 0.2956
loss in 595 batch is 3.0265
loss in 630 batch is 1.0199
loss in 665 batch is 2.2584
loss in 700 batch is 1.8161
loss in 735 batch is 0.9925
loss in 770 batch is 2.3630
loss in 805 batch is 0.7308
loss in 840 batch is 0.3576
loss in 875 batch is 0.0839
7026
7026
val Loss: 1.3944 Acc: 0.7348

Epoch 9/9
----------
28100
loss in 35 batch is 0.5678
loss in 70 batch is 0.1312
loss in 105 batch is 0.6469
loss in 140 batch is 0.1249
loss in 175 batch is 0.1347
loss in 210 batch is 0.1496
loss in 245 batch is 0.3665
loss in 280 batch is 0.0128
loss in 315 batch is 0.1152
loss in 350 batch is 0.0215
loss in 385 batch is 0.0065
loss in 420 batch is 0.1495
loss in 455 batch is 0.0490
loss in 490 batch is 0.0206
loss in 525 batch is 0.0258
loss in 560 batch is 0.3046
loss in 595 batch is 0.0212
loss in 630 batch is 0.2952
loss in 665 batch is 0.2606
loss in 700 batch is 0.1636
loss in 735 batch is 0.2132
loss in 770 batch is 0.0215
loss in 805 batch is 0.0242
loss in 840

In [ ]:
# torch.save({
#             'epoch': 1,
#             'model_state_dict': model_toy.state_dict(),
#             'optimizer_state_dict': optimizer_ft.state_dict(),
#             }, './vgg19toy.checkpoint')

In [27]:
torch.save(model_ft,'resnet50_12epoch_model')

In [5]:
def get_input(idx):
        imgtemp = Image.open(images_train_list[idx])
#         if self.transform!=None :
#             imgtemp = self.transform(imgtemp)
        img = transforms.ToTensor()(imgtemp)
        fileName = images_train_list[idx].split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[trainLabels.loc[trainLabels.image==fileName ,'level'].values[0]] = 1
        return img ,imglabel

In [6]:
def get_pred_for_one_image(idx,model):
        test , test_label = get_input(idx)
        test = test.view(-1,3,224,224)
        test_label = test_label.view(-1,5)
        test = test.to(device)
        test_label = test_label.to(device)
        print("test_label",test_label)
        pred = model(test)
        print("pred",pred)

In [8]:
class_indices = {0: [] , 1: [] , 2: [] , 3 : [], 4:[]}
for idx in range(len(images_train_list)):
    fileName = images_train_list[idx].split('/')[-1].split('.')[0]
    label = torch.zeros(5,dtype=torch.long)
    label[trainLabels.loc[trainLabels.image==fileName ,'level'].values[0]] = 1
#     print(label)
    _, img_tru_class = torch.max(label,0)
#     print(img_tru_class)
    if idx%1000==0:
        print("idx ",idx)
    class_indices[img_tru_class.item()].append(idx)

idx  0
idx  1000
idx  2000
idx  3000
idx  4000
idx  5000
idx  6000
idx  7000
idx  8000
idx  9000
idx  10000
idx  11000
idx  12000
idx  13000
idx  14000
idx  15000
idx  16000
idx  17000
idx  18000
idx  19000
idx  20000
idx  21000
idx  22000
idx  23000
idx  24000
idx  25000
idx  26000
idx  27000
idx  28000


In [89]:
def visualize_index(idx,img_list):
    img = Image.open(img_list[idx])
    img.show()

In [11]:
images_train_list[class_indices[3][0]]

'./train/train_224/5789_left.tiff'

In [92]:
for i in range(5):
    index = class_indices[2][i]
    print("index",index)
    get_pred_for_one_image(class_indices[2][i],model_ft)
    visualize_index(class_indices[2][i],images_train_list)

index 3
test_label tensor([[0, 0, 1, 0, 0]], device='cuda:0')
pred tensor([[ 1.3853, -0.2005,  2.1212, -1.4302, -1.9960]], device='cuda:0',
       grad_fn=<AddmmBackward>)
index 4
test_label tensor([[0, 0, 1, 0, 0]], device='cuda:0')
pred tensor([[ 2.5631,  1.1130,  0.6820, -2.8097, -1.7097]], device='cuda:0',
       grad_fn=<AddmmBackward>)
index 10
test_label tensor([[0, 0, 1, 0, 0]], device='cuda:0')
pred tensor([[ 2.0091,  0.5335,  1.1184, -2.6434, -1.2409]], device='cuda:0',
       grad_fn=<AddmmBackward>)
index 15
test_label tensor([[0, 0, 1, 0, 0]], device='cuda:0')
pred tensor([[ 1.7963,  0.2341,  3.2350, -1.7537, -3.4278]], device='cuda:0',
       grad_fn=<AddmmBackward>)
index 24
test_label tensor([[0, 0, 1, 0, 0]], device='cuda:0')
pred tensor([[-5.3421, -4.9266,  4.3555,  3.5650,  2.3408]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [98]:
visualize_index(24,images_train_list)

In [99]:
images_train_list

['./train/train_224/35193_left.tiff',
 './train/train_224/23392_left.tiff',
 './train/train_224/2199_left.tiff',
 './train/train_224/2536_left.tiff',
 './train/train_224/43589_right.tiff',
 './train/train_224/3341_right.tiff',
 './train/train_224/23251_left.tiff',
 './train/train_224/20863_right.tiff',
 './train/train_224/10484_right.tiff',
 './train/train_224/5827_right.tiff',
 './train/train_224/3301_left.tiff',
 './train/train_224/6541_left.tiff',
 './train/train_224/13062_right.tiff',
 './train/train_224/27635_left.tiff',
 './train/train_224/10294_right.tiff',
 './train/train_224/36636_left.tiff',
 './train/train_224/19177_right.tiff',
 './train/train_224/2853_right.tiff',
 './train/train_224/7174_right.tiff',
 './train/train_224/3908_left.tiff',
 './train/train_224/19867_right.tiff',
 './train/train_224/9346_right.tiff',
 './train/train_224/22134_left.tiff',
 './train/train_224/5067_left.tiff',
 './train/train_224/1965_right.tiff',
 './train/train_224/35464_left.tiff',
 './train/t

In [113]:
f = open("train_idx", "w")

In [114]:
for file in images_train_list:
    fname = file.split("/")[-1]
    f.write(fname+"\n")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F